![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Fcurriculum-notebooks&branch=master&subPath=Science/WaterWellLocationsAlberta/water-well-locations-alberta.ipynb&depth=1" target="_parent"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"/></a>

# Water Well Locations in Alberta

The Alberta Goverment maintains a [database of water well locations in the province](https://www.alberta.ca/alberta-water-well-information-database-overview.aspx), containing "approximately 500,000 records with nearly 3,000 new records added annually."

This is a large dataset, and since it is in [Microsoft Access](https://github.com/mkleehammer/pyodbc) format it's not all that easy to import into a Jupyter notebook.

We have downloaded the dataset and exported the water well locations to a CSV file. `▶Run` the next cell to import the dataset.

In [ ]:
import pandas as pd
df = pd.read_csv('https://drive.google.com/u/0/uc?id=1ptQoyoGhAv81R4A7VzupiYBhbBSD5uEy&export=download', low_memory=False)
print('The dataset contains',df.shape[0],'records.')

## Map by Place Name

Let's generate a map to display the locations of these water wells.

It would be unmanageable to add that many markers to a map, but we can add markers for wells that are within half a degree of latitude or longitude of a location.

You should be able to change the `location` to any town or city in Alberta, then `▶Run` the cell to generate a map.

In [ ]:
location = 'Edmonton, AB'

# try to get the coordinates of that location
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='Callysto Water Wells')
try:
    coordinates = geolocator.geocode(location)
    latitude = coordinates.latitude
    longitude = coordinates.longitude
    print('Generating map of water wells near', location)
except:
    print('Location lookup unsuccessful, generating map of water wells near Lethbridge, AB')
    latitude = 49.7
    longitude = -112.8

# generate the map
import folium
from folium.plugins import MarkerCluster
m = folium.Map(location=[latitude,longitude],zoom_start=6)
mc = MarkerCluster()
latitudes = df['Latitude'].between(latitude-0.25, latitude+0.25)
longitudes = df['Longitude'].between(longitude-0.25, longitude+0.25)
map_data = df[(latitudes) & (longitudes)]
for row in map_data.itertuples():
    mc.add_child(folium.Marker(location=[row.Latitude,row.Longitude], popup=row.Well_ID))
m.add_child(mc)
m

## Map by Coordinates

If we would prefer to map the well locations near a particular set of coordinates (latitude and longitude), we can visit [OpenStreetMap](https://www.openstreetmap.org) and right-click on a location then click `Show address` to display the coordinates.

Or we can generate a map of Alberta with a grid and some markers using the next code cell to help us find coordinates.

In [ ]:
print('Click a marker on the map to see its coordinates')
m = folium.Map(location=[53.5,-114],zoom_start=5)
for lat in range(49, 60):
    for lon in range(-120, -110):
        A = (lat, lon)
        B = (lat, lon+1)
        C = (lat+1, lon+1)
        D = (lat+1, lon)
        box = [A, B, C, D, A]
        center = [lat+0.5, lon+0.5]
        folium.Marker(location=center, popup=center).add_to(m)
        folium.PolyLine(box).add_to(m)
m

Once we have latitude and longitude values, we can input them into the following code cell to see water wells near that location.

In [ ]:
latitude = 56.5
longitude = -111.5

m = folium.Map(location=[latitude,longitude],zoom_start=6)
mc = MarkerCluster()
latitudes = df['Latitude'].between(latitude-0.25, latitude+0.25)
longitudes = df['Longitude'].between(longitude-0.25, longitude+0.25)
map_data = df[(latitudes) & (longitudes)]
for row in map_data.itertuples():
    mc.add_child(folium.Marker(location=[row.Latitude,row.Longitude], popup=row.Well_ID))
m.add_child(mc)
m

## Counting Wells

We can also count how many water wells are in the dataset near a particular location.

In [ ]:
location = 'Edmonton, AB'
degrees_away = 0.5

coordinates = geolocator.geocode(location)
latitude = coordinates.latitude   # or put in a number here
longitude = coordinates.longitude # or put in a number here
da = degrees_away/2
latitudes = df['Latitude'].between(latitude-da, latitude+da)
longitudes = df['Longitude'].between(longitude-da, longitude+da)
map_data = df[(latitudes) & (longitudes)]
print('There are', map_data.shape[0], 'water wells within', degrees_away, 'degrees of', location)

## Mapping All Well Locations

If you want to generate a map of all water well locations in Alberta, copy the following code into the next code cell. This will take quite a while to run, and the resulting map may cause your web browser to freeze.

```python
m = folium.Map(location=[53.5,-114],zoom_start=5)
mc = MarkerCluster()
for row in df.itertuples():
    mc.add_child(folium.Marker(location=[row.Latitude,row.Longitude], popup=row.Well_ID))
m.add_child(mc)
m
```

## Conclusion

From this dataset of water wells we are able to generate maps of well locations across Alberta.

*Government of Alberta, Alberta Water Well Information Database. Retrieved 2020-12-19, from http://groundwater.alberta.ca/WaterWells/d. Contains Information licensed under the Open Government License – Alberta.*

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)